In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st


from fair.inverse import inverse_fair_scm
from tqdm.notebook import tqdm

from fair_spice.ensemble import run_ensemble
import fair_spice.config as config

In [2]:
SEEDS = config.RANDOM_SEEDS
SAMPLES = 1_000
F2XCO2_MEAN = 4.00
F2XCO2_NINETY = 0.48
NINETY_TO_ONESIGMA = st.norm.ppf(0.95)

# Create the parameter distributions to draw from


In [3]:
# the CO2 concentration [C] will be a static forcing in all the ensembles
# given by a 1% YoY growth:
nt = 141
Cpi = 284.32
C = Cpi*1.01**np.arange(nt)

# other model parameters will be drawn from uniform or normal distributions
r0_fs = st.uniform.rvs(loc=27.7, scale=41.3-27.7, random_state=SEEDS[10], size=SAMPLES)
rC_fs = st.uniform.rvs(loc=0, scale=0.0482, random_state=SEEDS[11], size=SAMPLES)
rT_fs = st.uniform.rvs(loc=-0.0847, scale=4.52+0.0847, random_state=SEEDS[12], size=SAMPLES)
pre_ind_co2 = st.norm.rvs(loc=277.147, scale=2.9, random_state=SEEDS[13], size=SAMPLES)

In [4]:
# ECS and TCR.
# we want to base the climate sensitivity on the parameter values observed in 
# coupled climate models. We'll load these from disk and numerically 
# calculate a multi-variate distribution based on these observations

tunings_file = config.ROOT_DIR / 'data_input/tunings/cmip6_twolayer_tuning_params.json'
column_order = ['q4x','lamg','cmix','cdeep','gamma_2l','eff']
tunings = config.load_tunings(tunings_file).drop(columns=['t4x'])[column_order]
tunings.head()

,q4x,lamg,cmix,cdeep,gamma_2l,eff
CESM2-WACCM-FV2,7.011729,-0.601681,8.170171,112.097270,0.704935,1.501194
E3SM-1-0,7.396112,-0.629308,8.393029,43.903583,0.363434,1.455885
NorESM2-LM,9.532072,-0.926445,5.604629,145.052415,0.819696,3.074719
CESM2-WACCM,7.856972,-0.705814,8.293804,89.669971,0.700155,1.525304
GISS-E2-2-G,7.192669,-1.642447,8.893610,411.847639,0.530129,0.651705


In [5]:
# Create a 6D Gaussian kernel based on the data from the model tunings
# and resample for the number of samples we want to cover
kde = st.gaussian_kde(tunings.T)
samples_raw = kde.resample(size=int(SAMPLES*1.25), seed=SEEDS[15])

samples = pd.DataFrame(samples_raw.T, columns=tunings.columns)
# remove unphysical combinations
samples[samples <= 0] = np.nan
# lamg is always < 0. But this truncnorm approx seems to overwrite
# sample distribution that came from the kde? TODO: check
samples.lamg = st.truncnorm.rvs(-2, 2, loc=-4/3, scale=0.5, size=int(SAMPLES*1.25), random_state=SEEDS[16])

samples = samples.dropna(how='any')[:SAMPLES]
samples.head()


,q4x,lamg,cmix,cdeep,gamma_2l,eff
0,7.497561,-1.156137,9.703151,95.942929,0.528074,0.645244
1,6.753141,-2.167808,8.279026,117.123042,0.685883,1.276127
2,8.528818,-1.293838,6.359235,43.292078,0.929985,1.467336
3,6.917942,-1.305245,7.778139,92.040589,0.487083,0.966554
4,9.179239,-0.987906,9.049638,49.697126,0.924632,1.548546


In [6]:
# convert the samples of the coupled-model observed parameters to ECS and TCR

f2x = st.norm.rvs(loc=F2XCO2_MEAN, scale=F2XCO2_NINETY/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[73])

ecs = -f2x/samples.lamg
tcr = f2x/(-samples.lamg + samples.gamma_2l*samples.eff)

In [7]:
# create all the configs as a dataframe and show the first 5 entries
configs = pd.DataFrame(dict(r0=r0_fs, rc=rC_fs, rt=rT_fs, F2x=f2x, tcr=tcr, ecs=ecs, other_rf=0.))
configs.head()

,r0,rc,rt,F2x,tcr,ecs,other_rf
0,34.237201,0.046983,4.098515,4.127057,2.757117,3.569695,0.0
1,32.167922,0.029863,0.468366,3.659703,1.202630,1.688204,0.0
2,37.653618,0.036696,0.100831,3.678141,1.383572,2.842813,0.0
3,32.630093,0.005103,3.940949,4.437090,2.498310,3.399431,0.0
4,33.919329,0.041117,2.489245,3.676911,1.519547,3.721924,0.0


## Run the model

We have created all the ensemble member configuration items. Now we can run
the model for each ensemble member.

`run_ensemble` uses a the python `multiprocessing` library to run on all
available CPUs. The output is saved to a netcdf file. See the [xarray_analysis
notebook](./xarray_analysis.ipynb) for examples of loading this data and
performing basic data analysis on it.

In [8]:
# create a iterator of dicts from the configs dataframe
iter_config = tqdm((x._asdict() for x in configs.itertuples(index=False)), total=SAMPLES)


def run_model(**config):
    # we want to record tcr and ecs separately, but the inverse_fair_scm model
    # expects them to be passed in together as an array. So we'll pop them out
    # and create the `tcrecs` object before running the function
    tcr = config.pop('tcr')
    ecs = config.pop('ecs')
    config['tcrecs'] = np.array([tcr, ecs])
    E, F, T = inverse_fair_scm(**config)
    # label the output as separate netcdf variables
    return {'emissions': E, 'forcing': F, 'temperature': T}

forcing = {'C': C}

run_ensemble(run_model, iter_config, forcing, outfile='ensemble.nc')

  0%|          | 0/1000 [00:00<?, ?it/s]